## 1. Setup and Installations

This section prepares the environment by cloning necessary repositories and installing Python packages.

In [ ]:
!git clone https://github.com/ludoplayer69/videotree
!git clone https://github.com/facebookresearch/perception_models.git
!git clone https://github.com/subhadarship/kmeans_pytorch

!pip install -q -U bitsandbytes accelerate transformers
!pip install -q gdown decord ftfy groq

## 2. Configuration

All key parameters for the pipeline are defined here. Modify this section to change model settings, paths, and hyperparameters for experiments.

In [ ]:
import os
from pathlib import Path
import torch

# Set your Groq API key here
# Get one from https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "  "

# To prevent CUDA memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

class PipelineConfig:
    # General
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MAX_VIDEOS_TO_PROCESS = 10  # Limit the number of videos for a quick run

    # Paths
    WORK_DIR = Path('/kaggle/working/')
    DATA_DIR = Path('/kaggle/input/egoschema/')
    
    # Derived Paths
    VIDEO_DIR = WORK_DIR / 'downloaded_videos'
    FRAMES_DIR = WORK_DIR / 'extracted_frames'
    FEATURES_DIR = WORK_DIR / 'extracted_features'
    OUTPUTS_DIR = WORK_DIR / 'outputs'
    
    # Data sources
    DRIVE_IDS_JSON = WORK_DIR / 'videotree/drive_ids.json'
    ANNOTATIONS_JSON = DATA_DIR / 'fullset_anno.json'
    CAPTIONS_JSON = DATA_DIR / 'blip2_fullset.json'

    # Feature Extraction
    FRAME_EXTRACTION_FPS = 1
    PERCEPTION_MODEL_NAME = 'PE-Core-B16-224'

    # Width Expansion (Adaptive Clustering)
    GROQ_MODEL = 'llama-3.1-8b-instant'
    MAX_CLUSTER_NUM = 32
    INIT_CLUSTER_NUM = 4
    RELEVANCE_THRESHOLD = 5  # Stop clustering if >= this many high-relevance frames are found

    # Depth Expansion (Hierarchical Clustering)
    NUM_SUBCLUSTERS = 4
    NUM_SUB_SUBCLUSTERS = 4

    # VLM Inference
    VLM_MODEL_ID = "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit"
    VLM_BATCH_SIZE = 10 # Lower if you encounter Out-of-Memory errors

    def __init__(self):
        # Create all necessary directories
        for path in [self.WORK_DIR, self.VIDEO_DIR, self.FRAMES_DIR, 
                     self.FEATURES_DIR, self.OUTPUTS_DIR]:
            path.mkdir(parents=True, exist_ok=True)

config = PipelineConfig()

## 3. Imports and Utility Functions

This section contains all required imports and helper functions used throughout the notebook.

In [2]:
import sys
import json
import cv2
import re
import gdown
import string
import time
from tqdm import tqdm
from PIL import Image
import numpy as np
from glob import glob
import gc

import torch
import torch.nn.functional as F
from transformers import AutoProcessor, AutoModelForVision2Seq
from groq import Groq
from scipy.cluster.hierarchy import linkage, fcluster

# Add cloned repos to system path for imports
sys.path.append(str(config.WORK_DIR / 'perception_models'))
sys.path.append(str(config.WORK_DIR / 'kmeans_pytorch'))
from kmeans_pytorch import kmeans

# --- Generic Utilities ---
def load_json(file_path: Path):
    with open(file_path, 'r') as f:
        return json.load(f)

def save_json(data, file_path: Path, indent=4):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=indent)
        
def _numeric_sort_key(p: Path):
    try:
        return int(p.stem)
    except ValueError:
        return p.stem

2025-08-23 04:32:25.595644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755923545.621196     259 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755923545.629207     259 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 4. Data Preparation

Downloads videos from Google Drive and loads the corresponding questions and captions.

In [3]:
def download_videos(drive_json_path: Path, save_path: Path, max_downloads: int):
    if not drive_json_path.exists():
        print(f"[ERROR] Drive IDs JSON not found at: {drive_json_path}")
        return
        
    drive_data = load_json(drive_json_path)
    video_ids_to_download = list(drive_data.items())[:max_downloads]
    
    for uuid, drive_id in tqdm(video_ids_to_download, desc="Downloading Videos"):
        output_file = save_path / f"{uuid}.mp4"
        if not output_file.exists(): # Skip if already downloaded
            gdown.download(id=drive_id, output=str(output_file), quiet=True)

def load_metadata(config: PipelineConfig):
    video_ids = [p.stem for p in config.VIDEO_DIR.glob("*.mp4")]
    annotations = load_json(config.ANNOTATIONS_JSON)
    captions = load_json(config.CAPTIONS_JSON)
    
    questions = {}
    for vid in video_ids:
        if vid not in annotations:
            continue
        
        anno = annotations[vid]
        option_keys = sorted([k for k in anno if k.startswith("option ")], key=lambda x: int(x.split()[1]))
        
        prompt = f"Question:\n{anno['question']}\n\nOptions:\n"
        for letter, key in zip(string.ascii_uppercase, option_keys):
            prompt += f"{letter}. {anno[key]}\n"
        prompt += "\nPlease choose the most appropriate answer (A–E)."
        questions[vid] = prompt
        
    return video_ids, questions, captions

# Execute Data Preparation
print("--- Starting Data Preparation ---")
download_videos(config.DRIVE_IDS_JSON, config.VIDEO_DIR, config.MAX_VIDEOS_TO_PROCESS)
video_ids, questions, captions = load_metadata(config)
print(f"Found {len(video_ids)} videos with corresponding questions.")

--- Starting Data Preparation ---


Found 10 videos with corresponding questions.


## 5. Module 1: Feature Extraction

Extracts frames from videos and computes feature embeddings using a pre-trained perception model.

In [4]:
class FeatureExtractor:
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = None
        self.preprocess = None

    def _load_model(self):
        if self.model is not None and self.preprocess is not None:
            return
        
        # The model code expects to be run from its own directory
        cwd = os.getcwd()
        os.chdir(self.config.WORK_DIR / 'perception_models')
        import core.vision_encoder.pe as pe
        import core.vision_encoder.transforms as transforms
        
        self.model = pe.CLIP.from_config(self.config.PERCEPTION_MODEL_NAME, pretrained=True).to(self.config.DEVICE).eval()
        self.preprocess = transforms.get_image_transform(self.model.image_size)
        
        os.chdir(cwd)
        print("Feature extraction model loaded.")

    def _extract_frames_for_video(self, video_path: Path):
        out_dir = self.config.FRAMES_DIR / video_path.stem
        out_dir.mkdir(exist_ok=True)

        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened(): return

        fps_ori = cap.get(cv2.CAP_PROP_FPS) or 30
        frame_interval = max(1, int(fps_ori // self.config.FRAME_EXTRACTION_FPS))
        
        count = 0
        success, img = cap.read()
        while success:
            if count % frame_interval == 0:
                cv2.imwrite(str(out_dir / f"{count}.jpg"), img)
            success, img = cap.read()
            count += 1
        cap.release()

    @torch.inference_mode()
    def _extract_features_for_dir(self, frames_dir: Path):
        if not frames_dir.exists(): return

        image_files = sorted(list(frames_dir.glob('*.jpg')), key=_numeric_sort_key)
        if not image_files: return

        feats_list = []
        for img_fp in image_files:
            img = Image.open(img_fp).convert('RGB')
            inp = self.preprocess(img).unsqueeze(0).to(self.config.DEVICE)
            with torch.cuda.amp.autocast():
                feat = self.model.encode_image(inp)
            feats_list.append(feat)
            
        stacked_feats = torch.cat(feats_list, dim=0)
        torch.save(stacked_feats, self.config.FEATURES_DIR / f"{frames_dir.name}.pt")

    def run(self, video_ids: list[str]):
        self._load_model()
        
        print("Extracting frames...")
        for video_id in tqdm(video_ids, desc="Extracting Frames"):
            video_path = self.config.VIDEO_DIR / f"{video_id}.mp4"
            self._extract_frames_for_video(video_path)

        print("Extracting features...")
        for video_id in tqdm(video_ids, desc="Extracting Features"):
            frames_dir = self.config.FRAMES_DIR / video_id
            self._extract_features_for_dir(frames_dir)

# Execute Feature Extraction
print("--- Starting Feature Extraction ---")
feature_extractor = FeatureExtractor(config)
feature_extractor.run(video_ids)

--- Starting Feature Extraction ---
Missing keys for loading model: []
Unexpected keys for loading model: []
Feature extraction model loaded.
Extracting frames...


Extracting Frames: 100%|██████████| 10/10 [00:40<00:00,  4.07s/it]


Extracting features...


Extracting Features:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_259/3067077015.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Extracting Features: 100%|██████████| 10/10 [00:38<00:00,  3.87s/it]


## 6. Module 2: Width Expansion (Adaptive Clustering)

Applies k-means clustering to the frame features. An LLM (via Groq) scores the relevance of representative frames to iteratively find the optimal number of clusters.

In [5]:
class WidthExpansion:
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = Groq(api_key=os.environ.get("GROQ_API_KEY"))

    def _get_relevance_scores(self, representative_frames, video_id, captions_dict, questions):
        # This method now expects a dictionary, which is handled in the calling function
        video_captions_list = captions_dict.get(video_id, [])
        question_text = questions.get(video_id, "")
        

        # Access captions by list index since we know it's a list
        descriptions = []
        for idx in representative_frames:
            caption = video_captions_list[idx] if 0 <= idx < len(video_captions_list) else 'No caption.'
            descriptions.append(f"Frame {idx}: {caption}")

        system_prompt = "You are an expert video analyst. Analyze frame descriptions and rate their relevance to answering the specific question."
        user_prompt = f"""VIDEO QUESTION: {question_text}\n
FRAME DESCRIPTIONS:\n{chr(10).join(descriptions)}\n
TASK: Rate each frame's relevance to answering the question on a scale of 1-3 (1=Not relevant, 2=Somewhat, 3=Highly relevant).\n
Provide ONLY the relevance scores in this exact format: frame relevance: [score1, score2, ...]\n
You must provide exactly {len(descriptions)} scores."""
        print('2')
        completion = self.model.chat.completions.create(
            model=self.config.GROQ_MODEL,
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
            temperature=0.0, max_tokens=1000
        )
        response_text = completion.choices[0].message.content
        
        match = re.search(r'\[([0-9,\s]+)\]', response_text)
        if match:
            scores = [int(x.strip()) for x in match.group(1).split(',') if x.strip().isdigit()]
            return scores, completion.usage.total_tokens
        return [], 0


    def run_for_video(self, video_id: str, questions: dict, captions: list): # Expects a list of tuples
        features_path = self.config.FEATURES_DIR / f"{video_id}.pt"
        if not features_path.exists():
            print(f"[WARN] Features not found for {video_id}, skipping.")
            return

        # --- FIX: Convert the list of tuples into a dictionary ---
        captions_dict = dict(captions)
        # --- END FIX ---

        frame_feats = torch.load(features_path).to(self.config.DEVICE)
        num_clusters = self.config.INIT_CLUSTER_NUM
        all_attempts = []

        while num_clusters <= self.config.MAX_CLUSTER_NUM:
            cluster_ids, cluster_centers = kmeans(X=frame_feats, num_clusters=num_clusters, distance='cosine', device=self.config.DEVICE)
            
            cluster_centers = cluster_centers.to(self.config.DEVICE)

            representative_frames = []
            for i in range(num_clusters):
                cluster_indices = torch.where(cluster_ids == i)[0]
                if len(cluster_indices) > 0:
                    distances = torch.norm(frame_feats[cluster_indices] - cluster_centers[i], dim=1)
                    closest_local_idx = torch.argmin(distances).item()
                    representative_frames.append(cluster_indices[closest_local_idx].item())
            
            representative_frames = sorted(list(set(representative_frames)))
            # Pass the newly created dictionary to the helper function
            scores, tokens = self._get_relevance_scores(representative_frames, video_id, captions_dict, questions)
            
            high_relevance_count = scores.count(3)
            attempt_data = {
                'num_clusters': num_clusters,
                'representative_frames': representative_frames,
                'cluster_assignments': cluster_ids.tolist(),
                'frame_relevance': scores,
                'high_relevance_count': high_relevance_count,
                'tokens_used': tokens
            }
            all_attempts.append(attempt_data)
            
            if high_relevance_count >= self.config.RELEVANCE_THRESHOLD:
                break
            num_clusters *= 2
        
        if not all_attempts:
            print(f"[WARN] Could not perform clustering for {video_id}.")
            return

        final_result = all_attempts[-1]
        output_data = {
            'video_id': video_id,
            'final_result': final_result,
            'all_clustering_attempts': all_attempts,
            'total_tokens': sum(a['tokens_used'] for a in all_attempts)
        }
        save_json(output_data, self.config.OUTPUTS_DIR / f'width_expansion_{video_id}.json')
        return output_data

# Execute Width Expansion
print("--- Starting Width Expansion (Adaptive Clustering) ---")
width_expander = WidthExpansion(config)
for video_id in tqdm(video_ids, desc="Processing Videos (Width)"):
    # The 'captions' variable here is the original list of tuples
    width_expander.run_for_video(video_id, questions, captions)
print("Width expansion complete.")

--- Starting Width Expansion (Adaptive Clustering) ---


Processing Videos (Width):   0%|          | 0/10 [00:00<?, ?it/s]

running k-means on cuda..



[running kmeans]: 0it [00:00, ?it/s]
[running kmeans]: 0it [00:00, ?it/s, center_shift=188.140259, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00,  3.90it/s, center_shift=188.140259, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00,  3.90it/s, center_shift=6.656952, iteration=2, tol=0.000100]  
[running kmeans]: 2it [00:00,  3.90it/s, center_shift=2.033382, iteration=3, tol=0.000100]
[running kmeans]: 3it [00:00,  3.90it/s, center_shift=0.353337, iteration=4, tol=0.000100]
[running kmeans]: 4it [00:00,  3.90it/s, center_shift=0.139455, iteration=5, tol=0.000100]
[running kmeans]: 6it [00:00, 21.00it/s, center_shift=0.000000, iteration=6, tol=0.000100]


2
running k-means on cuda..


...

running k-means on cuda..



[running kmeans]: 0it [00:00, ?it/s]
[running kmeans]: 0it [00:00, ?it/s, center_shift=5081.288574, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 68.49it/s, center_shift=42.789051, iteration=2, tol=0.000100]
[running kmeans]: 2it [00:00, 91.61it/s, center_shift=0.343911, iteration=3, tol=0.000100] 
[running kmeans]: 3it [00:00, 102.72it/s, center_shift=2.425100, iteration=4, tol=0.000100]
[running kmeans]: 5it [00:00, 128.80it/s, center_shift=0.000000, iteration=5, tol=0.000100]

2



Processing Videos (Width): 100%|██████████| 10/10 [03:38<00:00, 21.83s/it]

Width expansion complete.


## 7. Module 3: Depth Expansion (Hierarchical Clustering)

Uses the relevance scores from the previous step to guide a hierarchical clustering process. High-relevance clusters are broken down into more granular sub-clusters to provide more detail to the VLM.

In [6]:
class DepthExpansion:
    def __init__(self, config: PipelineConfig):
        self.config = config

    def _hierarchical_clustering(self, features, cluster_ids, relevance_scores):
        clusters = {i: {} for i in range(max(cluster_ids) + 1)}

        for cid in set(cluster_ids):
            score = relevance_scores[cid] if cid < len(relevance_scores) else 1
            indices = [i for i, x in enumerate(cluster_ids) if x == cid]
            if len(indices) < 2 or score == 1:
                clusters[cid] = [indices]
                continue

            sub_features = features[indices]
            linked_sub = linkage(sub_features.cpu().numpy(), method='ward')
            sub_labels = fcluster(linked_sub, self.config.NUM_SUBCLUSTERS, criterion='maxclust') - 1
            
            if score == 2:
                clusters[cid] = [[indices[j] for j in np.where(sub_labels == i)[0]] for i in range(self.config.NUM_SUBCLUSTERS)]
                continue
            
            # Score == 3: Perform sub-subclustering
            sub_sub_clusters = []
            for sub_cid in range(self.config.NUM_SUBCLUSTERS):
                sub_indices = np.where(sub_labels == sub_cid)[0]
                if len(sub_indices) < 2:
                    sub_sub_clusters.append([indices[i] for i in sub_indices])
                    continue
                
                subsub_features = sub_features[sub_indices]
                linked_subsub = linkage(subsub_features.cpu().numpy(), method='ward')
                subsub_labels = fcluster(linked_subsub, self.config.NUM_SUB_SUBCLUSTERS, criterion='maxclust') - 1
                for subsub_cid in range(self.config.NUM_SUB_SUBCLUSTERS):
                    final_indices = sub_indices[np.where(subsub_labels == subsub_cid)[0]]
                    sub_sub_clusters.append([indices[i] for i in final_indices])
            clusters[cid] = sub_sub_clusters
        return clusters

    def _find_closest_points(self, features, clusters):
        final_indices = set()
        for primary_cluster in clusters.values():
            for sub_cluster in primary_cluster:
                if not sub_cluster: continue
                points = features[torch.tensor(sub_cluster, dtype=torch.long)]
                centroid = points.mean(dim=0)
                distances = torch.norm(points - centroid, dim=1)
                closest_idx = torch.argmin(distances).item()
                final_indices.add(sub_cluster[closest_idx])
        return sorted(list(final_indices))

    def run_for_video(self, video_id: str):
        width_results_path = self.config.OUTPUTS_DIR / f'width_expansion_{video_id}.json'
        features_path = self.config.FEATURES_DIR / f"{video_id}.pt"
        if not width_results_path.exists() or not features_path.exists():
            return

        width_data = load_json(width_results_path)['final_result']
        features = torch.load(features_path)

        clusters = self._hierarchical_clustering(
            features, width_data['cluster_assignments'], width_data['frame_relevance'])
        
        final_frames = self._find_closest_points(features, clusters)

        output_data = {
            "video_id": video_id,
            "original_representative_frames": width_data['representative_frames'],
            "final_representative_frames": final_frames
        }
        save_json(output_data, self.config.OUTPUTS_DIR / f'depth_expansion_{video_id}.json')
        return output_data

# Execute Depth Expansion
print("--- Starting Depth Expansion (Hierarchical Clustering) ---")
depth_expander = DepthExpansion(config)
for video_id in tqdm(video_ids, desc="Processing Videos (Depth)"):
    depth_expander.run_for_video(video_id)
print("Depth expansion complete.")

--- Starting Depth Expansion (Hierarchical Clustering) ---


Processing Videos (Depth): 100%|██████████| 10/10 [00:00<00:00, 118.44it/s]

Depth expansion complete.


In [7]:
import os
import json
import torch
import torch.nn.functional as F

# --- Config ---
features_dir = "/kaggle/working/extracted_features"
json_dir = "/kaggle/working/outputs"
output_dir = "/kaggle/working/filtered_jsons_thresholded"
os.makedirs(output_dir, exist_ok=True)

threshold = 0.8  # cosine similarity threshold

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

# --- Loop over all .pt feature files ---
for feat_file in os.listdir(features_dir):
    if not feat_file.endswith(".pt"):
        continue

    # Extract video ID from filename
    video_id = os.path.splitext(feat_file)[0]  # e.g. "0074f737-11cb-497d-8d07-77c3a8127391"

    # Load features
    feat_path = os.path.join(features_dir, feat_file)
    frame_features = torch.load(feat_path)  # shape: [num_frames, 1024]

    # Load corresponding JSON
    json_filename = f"depth_expansion_{video_id}.json"
    json_path = os.path.join(json_dir, json_filename)
    if not os.path.exists(json_path):
        print(f"⚠️ JSON not found for {video_id}, skipping...")
        continue

    data = load_json(json_path)
    indices = data.get("final_representative_frames", [])

    # --- Apply threshold filtering ---
    filtered_indices = []
    if indices:
        filtered_indices.append(indices[0])
        ref_idx = indices[0]

        for next_idx in indices[1:]:
            sim = F.cosine_similarity(
                frame_features[ref_idx].unsqueeze(0),
                frame_features[next_idx].unsqueeze(0)
            ).item()

            if sim < threshold:
                filtered_indices.append(next_idx)

            ref_idx = next_idx  # update reference

    # --- Save new JSON ---
    out_data = {"final_representative_frames": filtered_indices}
    out_path = os.path.join(output_dir, f"{video_id}_thresholded.json")
    with open(out_path, "w") as f:
        json.dump(out_data, f, indent=2)

    print(f"✅ Processed {video_id} → saved {out_path}")


✅ Processed 00f93e1e-cf4e-4835-88b4-4ad68216e86f → saved /kaggle/working/filtered_jsons_thresholded/00f93e1e-cf4e-4835-88b4-4ad68216e86f_thresholded.json
✅ Processed 0074f737-11cb-497d-8d07-77c3a8127391 → saved /kaggle/working/filtered_jsons_thresholded/0074f737-11cb-497d-8d07-77c3a8127391_thresholded.json
✅ Processed 01a144a5-24d2-4a5a-af01-1f318d674bed → saved /kaggle/working/filtered_jsons_thresholded/01a144a5-24d2-4a5a-af01-1f318d674bed_thresholded.json
✅ Processed 02925d7a-a5db-4127-8c31-b232e78b684d → saved /kaggle/working/filtered_jsons_thresholded/02925d7a-a5db-4127-8c31-b232e78b684d_thresholded.json
✅ Processed 03657401-d4a4-40d0-9b03-d7e093ef93d1 → saved /kaggle/working/filtered_jsons_thresholded/03657401-d4a4-40d0-9b03-d7e093ef93d1_thresholded.json
✅ Processed 00b9a0de-c59e-49cb-a127-6081e2fb8c8e → saved /kaggle/working/filtered_jsons_thresholded/00b9a0de-c59e-49cb-a127-6081e2fb8c8e_thresholded.json
✅ Processed 026a2f15-c454-4c28-80e0-24c85d7f4ecf → saved /kaggle/working/fil

In [8]:
f = load_json('/kaggle/working/filtered_jsons_thresholded/0074f737-11cb-497d-8d07-77c3a8127391_thresholded.json')
f

{'final_representative_frames': [3,
  15,
  20,
  32,
  36,
  40,
  44,
  63,
  65,
  70,
  71,
  73,
  100,
  105,
  122,
  154,
  165,
  177]}

## 8. Module 4: VLM Question Answering

Loads a powerful Vision-Language Model. It processes the question along with the refined set of keyframes from the depth expansion step to generate a final answer.

In [9]:
import re
from collections import Counter

def most_frequent_choice(text, letter_to_num=None):
    if letter_to_num is None:
        letter_to_num = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5}

    # Extract lines and letters
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    letters = []
    for ln in lines:
        m = re.match(r'^([A-Z])\.', ln, flags=re.I)
        if m:
            letters.append(m.group(1).upper())

    # Find most frequent letter
    if not letters:
        return None, None

    most_letter, freq = Counter(letters).most_common(1)[0]
    most_num = letter_to_num.get(most_letter)

    return most_num

In [10]:
#load answer
subest_json_path = '/kaggle/working/videotree/subset_answers.json'
answer = load_json(subest_json_path)
answer['0074f737-11cb-497d-8d07-77c3a8127391']

3

In [11]:
class VLMInference:
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = None
        self.processor = None

    def _load_model(self):
        if self.model is not None and self.processor is not None:
            return
        
        self.processor = AutoProcessor.from_pretrained(self.config.VLM_MODEL_ID, trust_remote_code=True)
        self.model = AutoModelForVision2Seq.from_pretrained(
            self.config.VLM_MODEL_ID,
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            device_map="auto"
        ).eval()
        print("VLM model loaded.")

    def clear_memory(self):
        del self.model
        gc.collect()
        torch.cuda.empty_cache()
        

    def _extract_frames_at_1fps(self, video_path: Path) -> dict[int, Image.Image]:
        cap = cv2.VideoCapture(str(video_path))
        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        
        frames = {}
        frame_idx, saved_idx = 0, 0
        while True:
            ret, frame = cap.read()
            if not ret: break
            if frame_idx % int(fps) == 0:
                frames[saved_idx] = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                saved_idx += 1
            frame_idx += 1
        cap.release()
        return frames

    def run_for_video(self, video_id: str, question: str):
        self._load_model()
        # depth_results_path = self.config.OUTPUTS_DIR / f'depth_expansion_{video_id}.json'
        t_out = "/kaggle/working/filtered_jsons_thresholded"
        depth_results_path = os.path.join(t_out, f"{video_id}_thresholded.json")
        video_path = self.config.VIDEO_DIR / f"{video_id}.mp4"

        # if not depth_results_path.exists():
        #     print(f"[WARN] Depth expansion results not found for {video_id}")
        #     return None
            
        depth_data = load_json(depth_results_path)
        keyframe_indices = depth_data['final_representative_frames']
        
        # The indices from feature extraction correspond to frames extracted at ~1 FPS
        all_frames = self._extract_frames_at_1fps(video_path)
        keyframe_images = [all_frames[i] for i in keyframe_indices if i in all_frames]
        
        if not keyframe_images:
            print(f"[WARN] No keyframe images found for {video_id}")
            return None
            
        # Batch processing for memory efficiency
        answers = []
        for i in range(0, len(keyframe_images), self.config.VLM_BATCH_SIZE):
            batch_images = keyframe_images[i:i + self.config.VLM_BATCH_SIZE]
            content = [{"type": "image", "image": img} for img in batch_images]
            content.append({"type": "text", "text": question})
            messages = [{"role": "user", "content": content}]
            
            try:
                inputs = self.processor.apply_chat_template(
                    messages, add_generation_prompt=True, tokenize=True,
                    return_dict=True, return_tensors="pt"
                ).to(self.model.device)

                with torch.no_grad():
                    outputs = self.model.generate(**inputs, max_new_tokens=50)
                
                answer = self.processor.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
                answers.append(answer.strip())

            except torch.cuda.OutOfMemoryError:
                print(f"[WARN] CUDA OOM on video {video_id}, batch {i}. Skipping batch.")
            finally:
                torch.cuda.empty_cache()
                
        return "\n".join(answers)


# Execute VLM Inference and Display Final Results
print("--- Starting VLM Question Answering ---")
vlm = VLMInference(config)
correct_count = 0
total_count = 0

for video_id in video_ids:
    print(f"\n{'='*20} Final Result for Video: {video_id} {'='*20}")
    question_text = questions.get(video_id, "No question available.")
    final_answer = vlm.run_for_video(video_id, question_text)
    choices = most_frequent_choice(final_answer)
    print("Most common letter(s):", choices)

    ground_truth = answer[video_id] + 1

    # Count total and correct answers
    total_count += 1
    if ground_truth ==choices:
        correct_count += 1

    print(f"\n❓ QUESTION:\n{question_text}")
    print(f"\n🧠 MODEL ANSWER:\n{final_answer if final_answer else 'Could not generate an answer.'}")
    print(f"\n✅ Ground Truth Answer:\n{ground_truth}")

# Final report
accuracy = (correct_count / total_count * 100) if total_count > 0 else 0
print("\n--- Pipeline Finished ---")
print(f"✅ Correct Answers: {correct_count}/{total_count} ({accuracy:.2f}% accuracy)")

--- Starting VLM Question Answering ---

==================== Final Result for Video: 00b9a0de-c59e-49cb-a127-6081e2fb8c8e ====================


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


VLM model loaded.
Most common letter(s): 5

❓ QUESTION:
Question:
What was the primary purpose of the cup of water in this video, and how did it contribute to the overall painting process?

Options:
A. To provide a source of water for the paintbrush.
B. To provide a place to store the paintbrush.
C. To provide a place to dispose of the paintbrush.
D. To provide a place to rest the paintbrush.
E. To clean the paintbrush.

Please choose the most appropriate answer (A–E).

🧠 MODEL ANSWER:
E. To clean the paintbrush.

✅ Ground Truth Answer:
5

==================== Final Result for Video: 0074f737-11cb-497d-8d07-77c3a8127391 ====================
Most common letter(s): 4

❓ QUESTION:
Question:
Taking into account all the actions performed by c, what can you deduce about the primary objective and focus within the video content?

Options:
A. C is cooking.
B. C is doing laundry.
C. C is cleaning the kitchen.
D. C is cleaning dishes.
E. C is cleaning the bathroom.

Please choose the most appropr